## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../weatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Zaraza,VE,9.3503,-65.3245,75.33,77,4,9.71,clear sky,2022-02-05 01:39:04
1,1,Benguela,AO,-12.5763,13.4055,73.94,73,100,4.94,overcast clouds,2022-02-05 01:36:40
2,2,Tasiilaq,GL,65.6145,-37.6368,24.96,69,100,42.73,overcast clouds,2022-02-05 01:34:37
3,3,Qaanaaq,GL,77.4840,-69.3632,1.17,70,100,6.51,overcast clouds,2022-02-05 01:34:47
4,4,Bethel,US,41.3712,-73.4140,31.50,84,100,8.05,overcast clouds,2022-02-05 01:39:05


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Zaraza,VE,9.3503,-65.3245,75.33,77,4,9.71,clear sky,2022-02-05 01:39:04
6,6,Rikitea,PF,-23.1203,-134.9692,79.05,68,4,7.18,clear sky,2022-02-05 01:39:06
10,10,Atuona,PF,-9.8000,-139.0333,78.37,77,25,15.90,scattered clouds,2022-02-05 01:39:07
15,15,Vaini,TO,-21.2000,-175.2000,78.96,94,100,17.27,overcast clouds,2022-02-05 01:37:47
18,18,Kapaa,US,22.0752,-159.3190,79.12,79,40,1.01,light rain,2022-02-05 01:39:09
19,19,Hilo,US,19.7297,-155.0900,78.53,79,100,9.22,overcast clouds,2022-02-05 01:37:07
24,24,Olinda,BR,-8.0089,-34.8553,82.13,65,40,6.91,scattered clouds,2022-02-05 01:39:11
25,25,Saint-Philippe,RE,-21.3585,55.7679,81.99,89,100,25.64,moderate rain,2022-02-05 01:39:11
31,31,Manaus,BR,-3.1019,-60.0250,75.69,100,75,1.14,light rain,2022-02-05 01:39:12
32,32,Sola,VU,-13.8833,167.5500,82.51,77,100,21.65,overcast clouds,2022-02-05 01:39:13


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                133
City                   133
Country                133
Lat                    133
Lng                    133
Max Temp               133
Humidity               133
Cloudiness             133
Wind Speed             133
Current Description    133
Date                   133
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Zaraza,VE,75.33,clear sky,9.3503,-65.3245,
6,Rikitea,PF,79.05,clear sky,-23.1203,-134.9692,
10,Atuona,PF,78.37,scattered clouds,-9.8000,-139.0333,
15,Vaini,TO,78.96,overcast clouds,-21.2000,-175.2000,
18,Kapaa,US,79.12,light rain,22.0752,-159.3190,
19,Hilo,US,78.53,overcast clouds,19.7297,-155.0900,
24,Olinda,BR,82.13,scattered clouds,-8.0089,-34.8553,
25,Saint-Philippe,RE,81.99,moderate rain,-21.3585,55.7679,
31,Manaus,BR,75.69,light rain,-3.1019,-60.0250,
32,Sola,VU,82.51,overcast clouds,-13.8833,167.5500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]  
    
    params["location"] = f"{lat}, {lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Zaraza,VE,75.33,clear sky,9.3503,-65.3245,Hotel Zaraza
6,Rikitea,PF,79.05,clear sky,-23.1203,-134.9692,People ThankYou
10,Atuona,PF,78.37,scattered clouds,-9.8000,-139.0333,Villa Enata
15,Vaini,TO,78.96,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Kapaa,US,79.12,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} F°</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))